## Librerias y directorios

### Importacion de librerias

In [1]:
# Bibliotecas para Manipulación de Datos y Análisis
import pandas as pd 
import numpy as np 

# Bibliotecas para Manejo de Archivos
import requests 
import zipfile  
import os  

# Visualización de Datos Perdidos
import missingno as msno

# Recolección y comprensión de los Datos





In [ ]:
path_csv_update = './data/bronze/shopping_behavior_updated.csv'
df = pd.read_csv(path_csv_update)

### Descubriendo los datos

### Descripcion y estadisticas basicas

Luego se analizaran visaulmente en la etapa de EDA

In [52]:
df.describe(include='all')

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,subscribed
count,41188.00000,41188,41188,41188,41188,41188,41188,41188,41188,41188,...,41188.000000,41188.000000,41188.000000,41188,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188
unique,NaN,12,4,8,3,3,3,2,10,5,...,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,2
top,NaN,admin.,married,university.degree,no,yes,no,cellular,may,thu,...,NaN,NaN,NaN,nonexistent,NaN,NaN,NaN,NaN,NaN,no
freq,NaN,10422,24928,12168,32588,21576,33950,26144,13769,8623,...,NaN,NaN,NaN,35563,NaN,NaN,NaN,NaN,NaN,36548
mean,40.02406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.567593,962.475454,0.172963,NaN,0.081886,93.575664,-40.502600,3.621291,5167.035911,NaN
std,10.42125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.770014,186.910907,0.494901,NaN,1.570960,0.578840,4.628198,1.734447,72.251528,NaN
min,17.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,0.000000,0.000000,NaN,-3.400000,92.201000,-50.800000,0.634000,4963.600000,NaN
25%,32.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,999.000000,0.000000,NaN,-1.800000,93.075000,-42.700000,1.344000,5099.100000,NaN
50%,38.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.000000,999.000000,0.000000,NaN,1.100000,93.749000,-41.800000,4.857000,5191.000000,NaN
75%,47.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.000000,999.000000,0.000000,NaN,1.400000,93.994000,-36.400000,4.961000,5228.100000,NaN


In [53]:
df.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
subscribed         object
dtype: object

### Extraccion de metadatos

**Datos de los clientes**

1 - age (numérico): edad

2 - job: tipo de empleo (categórico: ‘admin.’, ‘blue-collar’, ‘entrepreneur’, ‘housemaid’, ‘management’, ‘retired’, ‘self-employed’, ‘services’, ‘student’, ‘technician’, ‘unemployed’, ‘unknown’)

3 - marital: estado civil (categórico: ‘divorced’, ‘married’, ‘single’, ‘unknown’; nota: ‘divorced’ significa divorciado o viudo)

4 - education (categórico: ‘basic.4y’, ‘basic.6y’, ‘basic.9y’, ‘high.school’, ‘illiterate’, ‘professional.course’, ‘university.degree’, ‘unknown’): nivel educativo

5 - default: ¿tiene crédito en mora? (categórico: ‘no’, ‘yes’, ‘unknown’)

6 - housing: ¿tiene préstamo de vivienda? (categórico: ‘no’, ‘yes’, ‘unknown’)

7 - loan: ¿tiene préstamo personal? (categórico: ‘no’, ‘yes’, ‘unknown’)

Relacionado con el último contacto de la campaña actual:


8 - contact: tipo de comunicación del contacto (categórico: ‘cellular’, ‘telephone’)

9 - month: último mes de contacto del año (categórico: ‘jan’, ‘feb’, ‘mar’, …, ‘nov’, ‘dec’)

10 - day_of_week: último día de contacto de la semana (categórico: ‘mon’, ‘tue’, ‘wed’, ‘thu’, ‘fri’)

11 - duration: duración del último contacto, en segundos (numérico). Nota importante: este atributo afecta significativamente la variable objetivo (por ejemplo, si duration=0 entonces y=‘no’). Sin embargo, la duración no se conoce antes de realizar una llamada. Además, después de finalizar la llamada, y obviamente es conocido. Por lo tanto, esta variable solo debería incluirse con fines de referencia y debería descartarse si se busca un modelo predictivo realista.

**Otros atributos:**

12 - campaign: número de contactos realizados durante esta campaña para este cliente (numérico, incluye el último contacto)

13 - pdays: número de días que han pasado desde que el cliente fue contactado por última vez en una campaña anterior (numérico; 999 significa que el cliente no fue contactado previamente)

14 - previous: número de contactos realizados antes de esta campaña para este cliente (numérico)

15 - poutcome: resultado de la campaña de marketing anterior (categórico: ‘failure’, ‘nonexistent’, ‘success’)

**Atributos de contexto social y económico:**

16 - emp.var.rate: tasa de variación del empleo - indicador trimestral (numérico)

17 - cons.price.idx: índice de precios al consumidor - indicador mensual (numérico)

18 - cons.conf.idx: índice de confianza del consumidor - indicador mensual (numérico)

19 - euribor3m: tasa euribor a 3 meses - indicador diario (numérico)

20 - nr.employed: número de empleados - indicador trimestral (numérico)

**Variable de salida (objetivo deseado):**


21 - y - ¿el cliente ha suscrito un depósito a plazo? (binario: ‘yes’, ‘no’)

### Guardado de datasets

In [55]:
# guardo los dataframes con archivo .csv dentro de la carpeta "bronze"
path_csv_from_library = '../data/bronze/df_from_library.csv'
full_df_library.to_csv(path_csv_from_library, index=False)

path_csv_df_from_zip = '../data/bronze/df_from_zip.csv'
df_from_zip.to_csv(path_csv_df_from_zip, index=False)